In [ ]:
import turtle
import time

# -------------------- Settings --------------------
ANIMATION_SPEED = 0.05  # Speed of animation

# -------------------- Setup Window --------------------
wn = turtle.Screen()
wn.bgcolor("lightblue")
wn.title("Treasure Hunt - DFS")
wn.setup(1000, 600)

tile_size = 24
offset_x = -350
offset_y = 200

# -------------------- Classes --------------------
class Tile(turtle.Turtle):
    def __init__(self, color):
        super().__init__()
        self.shape("square")
        self.color(color)
        self.penup()
        self.speed(0)

class Explorer(turtle.Turtle):
    def __init__(self):
        super().__init__()
        self.shape("turtle")
        self.color("black")
        self.penup()
        self.speed(1)
        self.shapesize(stretch_wid=1.5, stretch_len=1.5)

# -------------------- Maps --------------------
map1 = [
    "WWWWWWWWWWWW",
    "WSLLLLLLLLLW",
    "WLLLLWLLLLLW",
    "WLLLLWLLLTWW",
    "WWWWWLLLLWWW",
    "WLLLLLLLLLLW",
    "WWWWWWWWWWWW"
]

map2 = [
    "WWWWWWWWWWWW",
    "WSLLLLLLLWLW",
    "WLWWWLWLWLWW",
    "WLLLLLWLWTLW",
    "WWWWWLWLWWWW",
    "WLLLLLLLLLLW",
    "WWWWWWWWWWWW"
]

map3 = [
    "WWWWWWWWWWWWWWWW",
    "WSLLLLLLLLLLLWLW",
    "WLLLLWLLLLWLLLW",
    "WLLLLWLLLTWLLLL",
    "WWWWWLLLLWWWWWW",
    "WLLLLLLLLLLLLLW",
    "WWWWWWWWWWWWWWWW"
]

# Choose map (1, 2, or 3)
map_choice = 3

if map_choice == 1:
    island = map1
elif map_choice == 2:
    island = map2
elif map_choice == 3:
    island = map3
else:
    island = map1

# -------------------- Initialization --------------------
walls = []
path = []
visited = set()
frontier = []
solution = {}
start_x = start_y = treasure_x = treasure_y = 0

# Tile instances
land = Tile("tan")
water = Tile("blue")
visited_tile = Tile("green")
frontier_tile = Tile("purple")
start_tile = Tile("red")
treasure_tile = Tile("yellow")
explorer = Explorer()

# -------------------- Draw Grid --------------------
def draw_grid(grid):
    global start_x, start_y, treasure_x, treasure_y
    for y in range(len(grid)):
        for x in range(len(grid[y])):
            ch = grid[y][x]
            screen_x = offset_x + x * tile_size
            screen_y = offset_y - y * tile_size

            if ch == 'W':
                water.goto(screen_x, screen_y)
                water.stamp()
                walls.append((screen_x, screen_y))
            elif ch == 'L':
                land.goto(screen_x, screen_y)
                land.stamp()
                path.append((screen_x, screen_y))
            elif ch == 'S':
                start_tile.goto(screen_x, screen_y)
                start_tile.stamp()
                start_x, start_y = screen_x, screen_y
                path.append((screen_x, screen_y))
            elif ch == 'T':
                treasure_tile.goto(screen_x, screen_y)
                treasure_tile.stamp()
                treasure_x, treasure_y = screen_x, screen_y
                path.append((screen_x, screen_y))

# -------------------- Explorer Movement --------------------
def move_explorer(x, y):
    explorer.goto(x + tile_size / 2, y - tile_size / 2)

# -------------------- DFS Search --------------------
def dfs_search(x, y):
    frontier.append((x, y))
    solution[(x, y)] = (x, y)

    while frontier:
        current = frontier.pop()
        if current in visited:
            continue
        visited.add(current)

        move_explorer(*current)
        visited_tile.goto(current)
        visited_tile.stamp()
        time.sleep(ANIMATION_SPEED)

        if current == (treasure_x, treasure_y):
            treasure_tile.goto(current)
            treasure_tile.stamp()
            return current

        neighbors = [
            (current[0] - tile_size, current[1]),
            (current[0], current[1] - tile_size),
            (current[0] + tile_size, current[1]),
            (current[0], current[1] + tile_size),
        ]

        for n in neighbors:
            if n in path and n not in visited:
                frontier.append(n)
                solution[n] = current
                frontier_tile.goto(n)
                frontier_tile.stamp()

    return None

# -------------------- Backtrack Path --------------------
def backtrack_path(end):
    x, y = end
    marker = Tile("orange")
    marker.shapesize(stretch_wid=1.5, stretch_len=1.5)
    marker.penup()
    marker.speed(0)
    marker.goto(x, y)
    marker.stamp()

    count = 0
    while (x, y) != (start_x, start_y):
        x, y = solution[(x, y)]
        marker.goto(x, y)
        marker.stamp()
        time.sleep(ANIMATION_SPEED / 2)
        count += 1
    return count

# -------------------- Run --------------------
draw_grid(island)
move_explorer(start_x, start_y)

start_time = time.time()
found = dfs_search(start_x, start_y)
end_time = time.time()

# -------------------- Output Result --------------------
writer = turtle.Turtle()
writer.hideturtle()
writer.penup()
writer.goto(-300, -250)

if found:
    path_len = backtrack_path(found)
    writer.color("darkgreen")
    writer.write(
        f"Treasure Found!\n\n"
        f"Total steps visited: {len(visited)}\n"
        f"Path length to treasure: {path_len}\n"
        f"Execution time: {round(end_time - start_time, 2)} seconds",
        font=("Arial", 16, "bold")
    )
else:
    writer.color("red")
    writer.write("Treasure not found!", font=("Arial", 18, "bold"))

wn.exitonclick()
